# Hedge Fund Holdings Analysis via EDGAR

## 1. Setup and Imports

In [7]:
from sec_edgar_downloader import Downloader
from bs4 import BeautifulSoup

import os
import pandas as pd

## 2. Download Latest 13F Filing (Tiger Global)

In [6]:
# Downloading the 13F Filings into a specific directory, for Tiger Global specifically.

dir = "sec"
email = "nair4986@gmail.com"
dl = Downloader(dir, email)  

tiger_global_cik = "0001167483"

dl.get("13F-HR", tiger_global_cik)

95

## 3. Parse and Structure Filing Data

In [11]:
# Locating the most recent filing out of all the Tiger Global filings in the directory
base_path = "sec-edgar-filings/0001167483/13F-HR"
filing_dates = sorted(os.listdir(base_path), reverse=True)
latest_filing = os.path.join(base_path, filing_dates[0], "full-submission.txt")

# Reading the contents of the file and storing it in memory as a string called 'content'
with open(latest_filing, "r", encoding="utf-8") as f:
    content = f.read()

# Each filing will store an XML section that contains all the holding data for the hedge fund
# This step extracts the XML data from the file we are parsing in
start_tag = "<informationTable>"
end_tag = "</informationTable>"

start_index = content.find(start_tag)
end_index = content.find(end_tag) + len(end_tag) 
xml_str = content[start_index:end_index]

# XML string is parsed and the rows are extracted
soup = BeautifulSoup(xml_str, "xml")
rows = soup.find_all("infoTable")

# Each holding is converted into a dictionary 
# The key data fields are extracted and we form Key=Value pairs from them.
data = []
for row in rows:
    data.append({
        "Name of Issuer": row.find("nameOfIssuer").text,
        "CUSIP": row.find("cusip").text,
        "Shares": int(row.find("sshPrnamt").text),
        "Share Type": row.find("sshPrnamtType").text,
        "Investment Discretion": row.find("investmentDiscretion").text,
        "Other Manager": row.find("otherManager").text if row.find("otherManager") else None,
        "Voting Authority (Sole)": row.find("Sole").text if row.find("Sole") else None,
    })

# Looping over all available submission files

In [13]:
# The same process as above is repeated, but through ALL possible filings within the SEC edgar filings.

filing_data = {}

for folder in sorted(os.listdir(base_path)):
    filing_path = os.path.join(base_path, folder, "full-submission.txt")

    if not os.path.exists(filing_path):
        continue 

    with open(filing_path, "r", encoding="utf-8") as f:
        content = f.read()

    start_tag = "<informationTable>"
    end_tag = "</informationTable>"

    start_index = content.find(start_tag)
    end_index = content.find(end_tag) + len(end_tag) 
    xml_str = content[start_index:end_index]

    soup = BeautifulSoup(xml_str, "xml")
    rows = soup.find_all("infoTable")

    holdings = []
    for row in rows:
        holdings.append({
        "Name of Issuer": row.find("nameOfIssuer").text,
        "CUSIP": row.find("cusip").text,
        "Shares": int(row.find("sshPrnamt").text),
        "Share Type": row.find("sshPrnamtType").text,
        "Investment Discretion": row.find("investmentDiscretion").text,
        "Other Manager": row.find("otherManager").text if row.find("otherManager") else None,
        "Voting Authority (Sole)": row.find("Sole").text if row.find("Sole") else None,
        })

    filing_data[folder] = holdings
